In [1]:
pip install rake-nltk

# Importing libraries

In [5]:
import numpy as np
import pandas as pd
from rake_nltk import Rake
import sklearn

In [6]:
dataset = pd.read_csv('../input/myanimelist-dataset-animes-profiles-reviews/animes.csv')
dataset.head()
dataset = dataset[0:15000]

# cleaning data and converting titles to lower case

In [7]:
syp = dataset['synopsis']
genre = dataset['genre']
title = dataset['title']
for i, t in enumerate(title):
    title[i] = t.lower()
    
title.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0             haikyuu!! second season
1             shigatsu wa kimi no uso
2                       made in abyss
3    fullmetal alchemist: brotherhood
4    kizumonogatari iii: reiketsu-hen
Name: title, dtype: object

In [8]:
syp = syp.fillna(' ')
for i,each_anime_genre in enumerate(genre):
    each_anime_genre = each_anime_genre.replace('[','')
    each_anime_genre = each_anime_genre.replace(']','')
    each_anime_genre = each_anime_genre.replace("'","")
    each_anime_genre = each_anime_genre.replace(",","")
    genre.iloc[i] = each_anime_genre
genre
    

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


0                       Comedy Sports Drama School Shounen
1                       Drama Music Romance School Shounen
2                   Sci-Fi Adventure Mystery Drama Fantasy
3        Action Military Adventure Comedy Drama Magic F...
4                      Action Mystery Supernatural Vampire
                               ...                        
14995                         Action Kids Adventure Comedy
14996                          Action Magic Fantasy Shoujo
14997                                Slice of Life Fantasy
14998                                       Sci-Fi Mystery
14999                        Action Adventure Mecha Sci-Fi
Name: genre, Length: 15000, dtype: object

# Extracting keywords

In [9]:
rake = Rake()
keywords = []
for plot in syp:
    rake.extract_keywords_from_text(plot)
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
dataset['keywords'] = keywords
dataset['keywords'][0]    
    

' large training camp alongside many notable volleyball teams karasuno high school volleyball team attempts volleyball team must learn overcome formidable opponents old class setter tooru oikawa standing rival nekoma high spring tournament instead senior players graduate national level players could possibly break archrival aoba jousai new — including toughest teams karasuno agrees new attacks would strengthen train harder take part powerful weapon mal rewrite last chance kageyama attempt also come high karasuno written world victory tokyo sturdiest skills sharpen settle refocus receive playing participation one moreover members may long japan invitation inter hope hope hinata following facing ever even efforts differences devise conquer blocks aiming'

In [10]:
dataset['keywords'][5896]

' short prologue ova manga volume vol counter measures ", capture evil spirits elsie receives dokurou skull branch chief task hell hell go escaped earth bundled 10'

In [11]:
dataset['bag of words'] = dataset['genre'] + dataset['keywords']

In [12]:
dataset['bag of words'][256]

'Action Sci-Fi Adventure Comedy Drama Shounen notorious robber named gasback legendary thief might appear heading towards macca city rumors say enormous influx bounty hunters bank heist 20 years $$ 300 000 prize 000 want vash stampede source meddling head collect causing area ann'

# Converting the bag of words into vectors

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = CountVectorizer()
vectorized_bag_of_words = vectorizer.fit_transform(dataset['bag of words'])
vectorized_bag_of_words = vectorized_bag_of_words.toarray()
#https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

In [24]:
vectorized_bag_of_words[1][3]

0

# Computing cosine similarity between the vectors and recommending the most simliar anime

In [25]:
def recommend(show_title, n_recom):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(dataset["title"] == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title 
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [26]:
recommend("made in abyss",8)

['pandora hearts',
 'made in abyss movie 3: fukaki tamashii no reimei',
 'tsumiki no ie',
 'dungeon ni deai wo motomeru no wa machigatteiru darou ka',
 'no.6',
 "angel's feather",
 'turn a gundam',
 'utawarerumono']

In [27]:
recommend("pandora hearts",3)

['oz no mahoutsukai (1986)', 'meng ta: xue mi cheng', 'made in abyss']

In [40]:
filtered = title[title=='trigun: badlands rumble']
print(filtered)
#series object

256     trigun: badlands rumble
5736    trigun: badlands rumble
Name: title, dtype: object


In [44]:
print(syp.iloc[256])
print(syp.iloc[5736])

 20 years after meddling into the bank heist of a notorious robber named Gasback, Vash the Stampede is heading towards Macca City. Rumors say that the legendary thief might appear there causing an enormous influx of bounty hunters in the area who want to collect the $$300,000,000 prize for his head. 
 
(Source: ANN)
 20 years after meddling into the bank heist of a notorious robber named Gasback, Vash the Stampede is heading towards Macca City. Rumors say that the legendary thief might appear there causing an enormous influx of bounty hunters in the area who want to collect the $$300,000,000 prize for his head. 
 
(Source: ANN)


In [47]:
filtered = title[title=='made in abyss']
print(filtered)

2       made in abyss
3079    made in abyss
Name: title, dtype: object


In [48]:
print(syp.iloc[2])
print(syp.iloc[3079])

The Abyss—a gaping chasm stretching down into the depths of the earth, filled with mysterious creatures and relics from a time long past. How did it come to be? What lies at the bottom? Countless brave individuals, known as Divers, have sought to solve these mysteries of the Abyss, fearlessly descending into its darkest realms. The best and bravest of the Divers, the White Whistles, are hailed as legends by those who remain on the surface. 
 
Riko, daughter of the missing White Whistle Lyza the Annihilator, aspires to become like her mother and explore the furthest reaches of the Abyss. However, just a novice Red Whistle herself, she is only permitted to roam its most upper layer. Even so, Riko has a chance encounter with a mysterious robot with the appearance of an ordinary young boy. She comes to name him Reg, and he has no recollection of the events preceding his discovery. Certain that the technology to create Reg must come from deep within the Abyss, the two decide to venture fort

In [49]:
dataset['bag of words'][2]

'Sci-Fi Adventure Mystery Drama Fantasy missing white whistle lyza create reg must come novice red whistle time long past ordinary young boy countless brave individuals gaping chasm stretching white whistles venture forth upper layer two decide true existence mysterious robot mysterious creatures mal rewrite harsh reality great pit furthest reaches fearlessly descending events preceding deep within darkest realms chance encounter become like abyss — reg come chasm abyss abyss abyss abyss written technology surface sought solve see roam riko riko remain relics recover recollection permitted name mysteries mother memories lies legends known know however however hailed filled eyes explore even earth divers divers discovery depths daughter comes certain bravest bottom bottom best aspires appearance annihilator'

In [50]:
dataset['bag of words'][256]

'Action Sci-Fi Adventure Comedy Drama Shounen notorious robber named gasback legendary thief might appear heading towards macca city rumors say enormous influx bounty hunters bank heist 20 years $$ 300 000 prize 000 want vash stampede source meddling head collect causing area ann'

In [52]:
vectorizer_test = CountVectorizer()
vectorized_bag_of_words_test = vectorizer_test.fit_transform(dataset['bag of words'])
vectorized_bag_of_words_test = vectorized_bag_of_words_test.toarray()

In [53]:
vectorized_bag_of_words_test[2]

array([0, 0, 0, ..., 0, 0, 0])

In [54]:
vectorized_bag_of_words_test[256]

array([0, 2, 0, ..., 0, 0, 0])

In [57]:
cosine_similarity(vectorized_bag_of_words_test[2],vectorized_bag_of_words_test[256],True)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.